# Imports

In [92]:
import pandas as pd

In [93]:
import xml.etree.ElementTree as et

In [94]:
import os
import sys

In [95]:
import matplotlib.pyplot as plt

In [96]:
#import argparse

# Define and parse command-line arguments
#parser = argparse.ArgumentParser(description='Process some integers.')
#parser.add_argument('--seed', type=int, help='Seed value')
#parser.add_argument('--traffic_scale', type=float, help='Traffic scale value')
#args = parser.parse_args()

# Functions

In [97]:
def get_last_folder(path):
    # Normalize the path to handle different separators and remove trailing separator
    normalized_path = os.path.normpath(path)
    # Split the path into components
    folders = normalized_path.split(os.sep)
    # Get the last folder
    last_folder = folders[-1]
    return last_folder

In [98]:
price_per_wh = 110 / 1000

In [99]:
def get_price(d_energy):
    return d_energy * price_per_wh

In [100]:
def transform_xml(xml_doc):
    attr = xml_doc.attrib
    for xml in xml_doc.iter('vehicle'):
        _dict = attr.copy()
        _dict.update(xml.attrib)
        
        yield _dict


In [101]:
def get_route_id(trip_id, trip_ids):
    trip_id = trip_id[:-2]
    line = trip_ids.loc[trip_ids['trip_id'] == trip_id]
    route_id = line.iloc[0]['route_id']

    return route_id

In [102]:
def get_routeids_df(base_folder):
    file_path = os.path.join(base_folder, "trips.txt")
    return pd.read_csv(file_path, sep=",")

In [103]:
def calc_elevation_up(group):
    z_diff = pd.to_numeric(group['z']).diff()

    # Filter out negative differences (upward movement)
    up = z_diff.apply(lambda x: x if x > 0 else 0)

    # Sum the positive differences to get the total upward movement
    total_up = up.sum()
    return total_up

In [104]:
def calc_elevation_down(group):
    z_diff = pd.to_numeric(group['z']).diff()

    # Filter out negative differences (upward movement)
    down = z_diff.apply(lambda x: x if x < 0 else 0)

    # Sum the positive differences to get the total upward movement
    total_down = down.sum()
    return total_down

In [105]:
def transform_xml_tripinfo(xml_doc):
    attr = xml_doc.attrib
    for xml in xml_doc.iter('tripinfo'):
        _dict = attr.copy()
        _dict.update(xml.attrib)
        
        yield _dict

In [106]:
def transform_xml_stops(xml_doc):
    for route in xml_doc.iter('route'):
        route_dict = route.attrib.copy()
        stops = []
        
        for stop in route.findall('stop'):
            stop_dict = stop.attrib.copy()
            stops.append(stop_dict)
        
        route_dict['stops'] = stops
        
        yield route_dict

In [107]:
def get_group_by_id(list_of_dfs, desired_id):
    for df in list_of_dfs:
        if desired_id in df['id'].values:
            return df[df['id'] == desired_id]
    raise ValueError(f"ID '{desired_id}' not found in any dataframe.")

In [108]:
def main(seed, scale):
    print(seed + " " + scale)

# XML to df

In [109]:
base_folder = "C:\\Users\\Admin\\Sumo\\nap_normafa_b"

In [110]:
file_path = os.path.join(base_folder, "emission.out.xml")
emission_output = et.parse(file_path)

transform = transform_xml(emission_output.getroot())
emission_output_list = list(transform)

emission_output_df = pd.DataFrame(emission_output_list)
emission_output_df = emission_output_df.drop(emission_output_df.columns[0], axis=1)

#emission_output_df.shape

In [111]:
file_path = os.path.join(base_folder, "Battery.out.xml")

battery_output = et.parse(file_path)
battery_output_root = battery_output.getroot()

transform = transform_xml(battery_output_root)
battery_output_list = list(transform)

battery_output_pd = pd.DataFrame(battery_output_list)

battery_output_pd = battery_output_pd.drop(battery_output_pd.columns[0], axis=1)
#battery_output_pd

In [112]:
file_path = os.path.join(base_folder, "tripinfo.xml")

b_tripinfo_output = et.parse(file_path)
b_tripinfo_output_root = b_tripinfo_output.getroot()

transform = transform_xml_tripinfo(b_tripinfo_output_root)
b_tripinfo_output_list = list(transform)

b_tripinfo_output_pd = pd.DataFrame(b_tripinfo_output_list)

b_tripinfo_output_pd = b_tripinfo_output_pd.drop(b_tripinfo_output_pd.columns[0], axis=1)
#b_tripinfo_output_pd

In [113]:
file_path = os.path.join(base_folder, "gtfs_pt_vehicles.add.xml")
stops = et.parse(file_path)

transform = transform_xml_stops(stops.getroot())
stops_list = list(transform)

stops_pd = pd.DataFrame(stops_list)

In [114]:
file_path = os.path.join(base_folder, "gtfs_pt_vehicles.add.xml")
vehicles = et.parse(file_path)

transform = transform_xml(vehicles.getroot())
vehicles_list = list(transform)

vehicles_pd = pd.DataFrame(vehicles_list)
vehicles_pd = vehicles_pd.drop(vehicles_pd.columns[0], axis=1)
#vehicles_pd

In [115]:
route_ids_df = get_routeids_df(base_folder)

## Grouping by id

In [116]:
grouped_df = battery_output_pd.groupby('id')

list_of_dfs = [group_data for _, group_data in grouped_df]
#list_of_dfs

In [117]:
grouped_emission_df = emission_output_df.groupby('id')

list_of_emission_dfs = [group_data for _, group_data in grouped_emission_df]
#list_of_emission_dfs



In [118]:
for group_id, group_data in grouped_df:
    avg_speed = group_data['speed'].astype(float).mean()
    
    energy = float(group_data['totalEnergyConsumed'].iloc[-1])-float(group_data['totalEnergyRegenerated'].iloc[-1])
    #print(get_price(energy))

In [119]:

results = []
for group_id, group_data in grouped_df:
    avg_speed = group_data['speed'].astype(float).mean()
    
    energy = get_price(float(group_data['totalEnergyConsumed'].iloc[-1])-float(group_data['totalEnergyRegenerated'].iloc[-1]))
    #print(float(group_data['totalEnergyConsumed'].iloc[-1])-float(group_data['totalEnergyRegenerated'].iloc[-1]))
    
    time_loss = b_tripinfo_output_pd.loc[b_tripinfo_output_pd['id'] == group_id, 'timeLoss'].values[0]
    route_length = b_tripinfo_output_pd.loc[b_tripinfo_output_pd['id'] == group_id, 'routeLength'].values[0]
    
    route = vehicles_pd.loc[vehicles_pd['id'] == group_id, 'route'].values[0]
    count_stops = stops_pd[stops_pd['id'] == route]['stops'].apply(len).sum()
    
    route_id = get_route_id(group_id, route_ids_df)
    
    z_up = calc_elevation_up(get_group_by_id(list_of_emission_dfs, group_id))
    z_down = calc_elevation_down(get_group_by_id(list_of_emission_dfs, group_id))
    
    # Store the results in a dictionary
    group_result = {
        'routeid': route_id,
        'id': group_id,
        'avgSpeed': avg_speed,
        'battery': energy,
        'timeloss': time_loss,
        'routeLength': route_length,
        'numOfStops': count_stops,
        'up': z_up,
        'down': z_down
    }
    
    # Append the dictionary to the results list
    results.append(group_result)

# Convert the results list to a DataFrame
result_df = pd.DataFrame(results)
#print(result_df)


# CSV

In [120]:
tableBattery = pd.read_csv('batteryData.csv', delimiter=';')

#tableBattery.shape

## settings

In [121]:
locSetting = get_last_folder(base_folder)
seedSetting = 'fgh'
trafficScaleSetting = 'asd'

## Df to csv

In [122]:
for index, row in result_df.iterrows():
    row_data = {
        'routeId': row['routeid'],
        'loc': locSetting,
        'tripId': row['id'],
        'seed': seedSetting,
        'avgSpeed': row['avgSpeed'],
        'timeloss': row['timeloss'],
        'route_length': row['routeLength'],
        'elevation_up': row['up'],
        'elevation_down': row['down'],
        'trafficScale': trafficScaleSetting,
        'numOfStops': row['numOfStops'],
        'emission': row['battery']
    }
    temp_df = pd.DataFrame([row_data])
    #any empty or all-NA columns in tableBattery are excluded before concatenating the DataFrames
    tableBattery = tableBattery.dropna(axis=1, how='all')

    tableBattery = pd.concat([tableBattery, temp_df], ignore_index=True)


'id': group_id,
        'avgSpeed': avg_speed,
        'batteryCapacity': energy,
        'timeloss': time_loss,
        'routeLength': route_length,
        'stops': count_stops

In [123]:
#tableBattery.to_csv('batteryData.csv', index=False, sep=';')
tableBattery

,routeId,loc,tripId,seed,avgSpeed,timeloss,route_length,elevation_up,elevation_down,trafficScale,numOfStops,emission
0,2400,nap_gellert_b,C6359838.0,1,1.013591,339.81,575.27,5.3400,-7.9300,0.3,2,69.618208
1,150,nap_gellert_b,C7246892.0,1,6.425740,33.46,660.1,2.3400,-2.3200,0.3,2,141.947410
2,270,nap_gellert_b,C734723628.0,1,5.872307,134.66,3647.97,105.6900,-68.5300,0.3,12,1252.215855
3,2120,nap_gellert_b,C745091034.0,1,3.558635,614.38,4075.61,22.7300,-79.8800,0.3,11,395.144354
4,2120,nap_gellert_b,C74509178.0,1,2.757995,1019.37,4018.91,80.8600,-24.3000,0.3,11,953.118991
...,...,...,...,...,...,...,...,...,...,...,...,...
1051,0050,nap_normafa_b,C78105129.0,fgh,3.472642,465.60,2667.11,14.1265,-34.5373,asd,8,463.788215
1052,0050,nap_normafa_b,C78105320.0,fgh,3.144238,542.33,2581.71,34.2404,-13.8806,asd,7,641.588970
1053,0050,nap_normafa_b,C78105332.0,fgh,3.493126,496.82,2581.71,33.9295,-13.5697,asd,7,691.294263
1054,0050,nap_normafa_b,C7810542.0,fgh,4.170306,324.09,2581.71,34.2162,-13.8564,asd,7,613.111653


In [124]:
if __name__ == "__main__":
    main(seed = 0, scale = 0)

TypeError: unsupported operand type(s) for +: 'int' and 'str'